In [1]:
import numpy as np
import tensorflow as tf

from model import *
from dataset import *

C:\Users\kjosev\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# set up options

options = {}
options['margin'] = 50000
options['lrate'] = 0.05
options['dim'] = 250
options['epochs'] = 50
options['batch_size'] = 256
options['dataset'] = 'awa2'
options['num_base_classes'] = 50
options['abs'] = True

In [3]:
# fixing the seed
np.random.seed(12345)
tf.set_random_seed(12345)

In [4]:
# HYPERNYM PREDICTION DATA
hypernyms, name2index = load_hypernym_dataset(options['dataset'], 
                                              options['num_base_classes'])

options['num_all_classes'] = len(name2index)

print('Number of base and parent classes:', len(name2index))
print('Number of hypernym pairs:', len(hypernyms))

Number of base and parent classes: 106
Number of hypernym pairs: 1332


In [5]:
# IMAGE CLASSIFICATION DATA
X, labels = load_data()
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, labels)

print('Dataset size:', X.shape)
print('Train size:', len(X_train))
print('Validation size:', len(X_val))
print('Test size:', len(X_test))

Dataset size: (37322, 4096)
Train size: 29857
Validation size: 3733
Test size: 3732


In [6]:
# Setting up our Adam optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate=options['lrate'])

In [7]:
# IMAGE CLASSIFICATION MODEL
im = tf.placeholder(tf.float64, shape=[None, X.shape[1]])
y = tf.placeholder(tf.int32, shape=[None])

classification_model = get_classification_model(im, y, options)

cls_acc, _, cls_loss, cls_pred = classification_model

flat_hit_pred = get_prediction(im, 10, options)

cls_train_op = optimizer.minimize(cls_loss)

In [8]:
# define our tf session
sess = tf.Session()


# initialize weights
init = tf.global_variables_initializer()
sess.run(init)
sess.run(tf.local_variables_initializer())

In [9]:
# Train the model end evaluate on our validation set

ic_loss_list = []
ic_train_acc_list = []
ic_val_acc_list = []

ic_steps = 0
for epoch in range(1, options['epochs'] + 1):
    print("Epoch:", epoch)
    
    # variables to keep track of our average loss and accuracy
    ic_loss_avg = 0
    train_acc_avg = 0

    # run batched training
    for batch_idxs in get_batch_idxs(len(X_train), options['batch_size']):
        
        _, accur, curr_cls_loss = sess.run(
            [cls_train_op, cls_acc, cls_loss], feed_dict= {
                im: X_train[batch_idxs], 
                y: y_train[batch_idxs]
            })
        
        ic_steps += 1
        
        # update average loss and accuracy
        ic_loss_avg += curr_cls_loss * len(batch_idxs) / float(len(X_train))
        train_acc_avg += accur * len(batch_idxs) / float(len(X_train))

        if ic_steps % 10 == 0:
            print("Steps: %05d Accuracy: %f" % (
                ic_steps, accur))

    print()
    print("Train Accuracy: %f Loss: %f\n" % (
            train_acc_avg, 
            ic_loss_avg
        )
    )
    
    ic_loss_list.append(ic_loss_avg)
    ic_train_acc_list.append(train_acc_avg)

    # variables to keep track of our average val loss and accuracy
    val_acc_avg = 0
    
    # get validation accuracy in batches
    for batch_idxs in get_batch_idxs(len(X_val), options['batch_size']):
        [curr_val_accur] = sess.run(
            [cls_acc], feed_dict={ 
                im: X_val[batch_idxs], 
                y: y_val[batch_idxs]
            }
        )

        val_acc_avg += curr_val_accur * len(batch_idxs) / float(len(X_val))

    print("\nValidation Accuracy: %f \n" % (val_acc_avg))
    
    ic_val_acc_list.append(val_acc_avg)

Epoch: 1
Steps: 00010 Accuracy: 0.078125
Steps: 00020 Accuracy: 0.082031
Steps: 00030 Accuracy: 0.074219
Steps: 00040 Accuracy: 0.050781
Steps: 00050 Accuracy: 0.042969
Steps: 00060 Accuracy: 0.058594
Steps: 00070 Accuracy: 0.054688
Steps: 00080 Accuracy: 0.054688
Steps: 00090 Accuracy: 0.035156
Steps: 00100 Accuracy: 0.031250
Steps: 00110 Accuracy: 0.062500

Train Accuracy: 0.054527 Loss: 588294731.382006


Validation Accuracy: 0.044736 

Epoch: 2
Steps: 00120 Accuracy: 0.046875
Steps: 00130 Accuracy: 0.046875
Steps: 00140 Accuracy: 0.035156
Steps: 00150 Accuracy: 0.046875
Steps: 00160 Accuracy: 0.035156
Steps: 00170 Accuracy: 0.089844
Steps: 00180 Accuracy: 0.121094
Steps: 00190 Accuracy: 0.093750
Steps: 00200 Accuracy: 0.097656
Steps: 00210 Accuracy: 0.097656
Steps: 00220 Accuracy: 0.105469
Steps: 00230 Accuracy: 0.078125

Train Accuracy: 0.073919 Loss: 445923425.892463


Validation Accuracy: 0.110367 

Epoch: 3
Steps: 00240 Accuracy: 0.113281
Steps: 00250 Accuracy: 0.101562
Steps: 

Steps: 02080 Accuracy: 0.921875
Steps: 02090 Accuracy: 0.886719
Steps: 02100 Accuracy: 0.886719

Train Accuracy: 0.897846 Loss: 9066075.262786


Validation Accuracy: 0.870881 

Epoch: 19
Steps: 02110 Accuracy: 0.878906
Steps: 02120 Accuracy: 0.890625
Steps: 02130 Accuracy: 0.894531
Steps: 02140 Accuracy: 0.871094
Steps: 02150 Accuracy: 0.890625
Steps: 02160 Accuracy: 0.910156
Steps: 02170 Accuracy: 0.906250
Steps: 02180 Accuracy: 0.914062
Steps: 02190 Accuracy: 0.890625
Steps: 02200 Accuracy: 0.910156
Steps: 02210 Accuracy: 0.933594
Steps: 02220 Accuracy: 0.878906

Train Accuracy: 0.902401 Loss: 8161257.348850


Validation Accuracy: 0.870346 

Epoch: 20
Steps: 02230 Accuracy: 0.878906
Steps: 02240 Accuracy: 0.925781
Steps: 02250 Accuracy: 0.910156
Steps: 02260 Accuracy: 0.929688
Steps: 02270 Accuracy: 0.902344
Steps: 02280 Accuracy: 0.914062
Steps: 02290 Accuracy: 0.929688
Steps: 02300 Accuracy: 0.925781
Steps: 02310 Accuracy: 0.917969
Steps: 02320 Accuracy: 0.917969
Steps: 02330 Accur

Steps: 04130 Accuracy: 0.949219
Steps: 04140 Accuracy: 0.945312
Steps: 04150 Accuracy: 0.988281
Steps: 04160 Accuracy: 0.941406
Steps: 04170 Accuracy: 0.960938
Steps: 04180 Accuracy: 0.949219
Steps: 04190 Accuracy: 0.976562
Steps: 04200 Accuracy: 0.964844
Steps: 04210 Accuracy: 0.957031

Train Accuracy: 0.958469 Loss: 2266098.496771


Validation Accuracy: 0.892848 

Epoch: 37
Steps: 04220 Accuracy: 0.937500
Steps: 04230 Accuracy: 0.949219
Steps: 04240 Accuracy: 0.972656
Steps: 04250 Accuracy: 0.964844
Steps: 04260 Accuracy: 0.968750
Steps: 04270 Accuracy: 0.949219
Steps: 04280 Accuracy: 0.945312
Steps: 04290 Accuracy: 0.980469
Steps: 04300 Accuracy: 0.984375
Steps: 04310 Accuracy: 0.960938
Steps: 04320 Accuracy: 0.914062

Train Accuracy: 0.961316 Loss: 2144089.130429


Validation Accuracy: 0.890437 

Epoch: 38
Steps: 04330 Accuracy: 0.953125
Steps: 04340 Accuracy: 0.968750
Steps: 04350 Accuracy: 0.945312
Steps: 04360 Accuracy: 0.972656
Steps: 04370 Accuracy: 0.980469
Steps: 04380 Accur